# 第12章：アンサンブル

## 12.2 決定木とロジスティック回帰のアンサンブル

### 12.2.2 モジュールなどの宣言

In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split # ホールドアウト用モジュール
from sklearn.metrics import accuracy_score, roc_auc_score # 評価指標用モジュール

### 12.2.3 csvファイルの読み込み

In [ ]:
# 決定木のsubmit用ファイルを読み込む
submit_dtc_df = pd.read_csv("../output/submit_dtc.csv", encoding="utf-8", dtype={"customer_id": str, "buy_proba": float})

# ロジスティック回帰のsubmit用ファイルを読み込む
submit_lgr_df = pd.read_csv("../output/submit_lgr.csv", encoding="utf-8", dtype={"customer_id": str, "buy_proba": float})

### 12.2.4 予測結果のマージ

In [ ]:
# 決定木とロジスティック回帰の予測結果をマージする
submit_dtc_lgr_df = pd.merge(submit_dtc_df, submit_lgr_df, on="customer_id", suffixes=["_dtc", "_lgr"])

# 行数・列数を確認する
print(f"submit_dtc_lgr_df.shape: {submit_dtc_lgr_df.shape}")

# DataFrameの中身を確認する
submit_dtc_lgr_df.head()

### 12.2.5 平均値の取得と分類

In [ ]:
# 平均を取得する
submit_dtc_lgr_df["buy_proba"] = submit_dtc_lgr_df[["buy_proba_dtc", "buy_proba_lgr"]].mean(axis=1)

In [ ]:
# 0.5をしきい値に分類を行う
submit_dtc_lgr_df["pred_buy_flag"] = np.where(submit_dtc_lgr_df["buy_proba"] >= 0.5, 1, 0)
print(submit_dtc_lgr_df.shape)
submit_dtc_lgr_df.head()

In [ ]:
submit_dtc_lgr_df["pred_buy_flag"].value_counts()

In [ ]:
submit_dtc_lgr_mean_df = submit_dtc_lgr_df[["customer_id", "buy_proba"]]
print(submit_dtc_lgr_mean_df.shape)
submit_dtc_lgr_mean_df.head()

In [ ]:
# outputディレクトリにsubmit用ファイルを出力する.
submit_dtc_lgr_mean_df.to_csv("../output/submit_dtc_lgr_mean.csv", encoding="utf-8", index=False)

## 12.3 LightGBM

### 12.3.2 pickleファイルの読み込みと訓練データの分割

In [ ]:
# 前処理済み中間データのdictを読み取る
pp_data_dict = joblib.load("../intermediate/pp_data_dict.pkl3")
# 辞書型変数の値に格納された決定木向け中間データを読み取る
dtc_train_df = pp_data_dict["dtc"]["train"]
dtc_test_df = pp_data_dict["dtc"]["test"]
# 6:4の割合でホールドアウト法を行う
dtc_train_train_df, dtc_train_valid_df = train_test_split(dtc_train_df, test_size=0.4, random_state=57, shuffle=True)
dtc_train_df.shape, dtc_train_train_df.shape, dtc_train_valid_df.shape, dtc_test_df.shape

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"
# 説明変数をfeaturesという変数に格納する
lgbc_features = dtc_train_df.columns.tolist()
# customer_idとbuy_flagは説明変数ではない為削除する
lgbc_features.remove("customer_id")
lgbc_features.remove("buy_flag")

### 12.3.3 データセットの生成とモデルの構築

In [ ]:
import lightgbm as lgb # LightGBM

In [ ]:
# LightGBM用データセットを生成する
lgb_train_train_dataset = lgb.Dataset(dtc_train_train_df[lgbc_features], dtc_train_train_df[target])
lgb_train_valid_dataset = lgb.Dataset(dtc_train_valid_df[lgbc_features], dtc_train_valid_df[target], reference=lgb_train_train_dataset)

lgb_train_dataset = lgb.Dataset(dtc_train_df[lgbc_features], dtc_train_df[target])
lgb_test_dataset = lgb.Dataset(dtc_test_df[lgbc_features], reference=lgb_train_dataset)

In [ ]:
# あらかじめ設定しておくハイパーパラメータ
lgbc_params = {
    "objective": "binary", # 問題設定: 二値分類
    "metric": "auc", # 評価関数: AUC
    "verbosity": -1 # 出力なし
}

In [ ]:
# 学習する
lgbc_clf = lgb.train(
    lgbc_params, # ハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    num_boost_round=100, # ブースティングを行う回数
    verbose_eval=20 # ブースティング20回につき1回結果を出力
)

### 12.3.4 予測と検証

In [ ]:
# train-validを予測する
lgbc_train_valid_proba_y = lgbc_clf.predict(dtc_train_valid_df[lgbc_features])
lgbc_train_valid_proba_y

In [ ]:
# train-validを検証する
lgbc_train_valid_auc_val = roc_auc_score(dtc_train_valid_df[target], lgbc_train_valid_proba_y)
lgbc_train_valid_auc_val

## 12.4 ハイパーパラメータチューニング

### 12.4.2 Optunaのインストールとimport

In [ ]:
import optuna.integration.lightgbm as optuna_lgb # OptunaによるLightGBM
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# LightGBM用データセットを生成する
lgb_train_train_dataset = lgb.Dataset(dtc_train_train_df[lgbc_features], dtc_train_train_df[target])
lgb_train_valid_dataset = lgb.Dataset(dtc_train_valid_df[lgbc_features], dtc_train_valid_df[target], reference=lgb_train_train_dataset)

lgb_train_dataset = lgb.Dataset(dtc_train_df[lgbc_features], dtc_train_df[target])
lgb_test_dataset = lgb.Dataset(dtc_test_df[lgbc_features], reference=lgb_train_dataset)

### 12.4.3 Optunaによる探索

In [ ]:
# OptunaによるLightGBMのハイパーパラメータチューニング
optuna_lgbc_clf = optuna_lgb.train(
    lgbc_params, # 固定のハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    valid_sets=lgb_train_valid_dataset, # 検証データ
    num_boost_round=100, # boostingを行う回数
    verbose_eval=20, # ブースティング20回につき1回結果を出力
    optuna_seed=57 # 再現性確保のためseed値を指定
)

In [ ]:
# 最適なパラメータの確認
best_lgbc_params = optuna_lgbc_clf.params
best_lgbc_params

## 12.5 最適化したLightGBMモデルの実装

### 12.5.1 モデルの構築

In [ ]:
# 学習する
lgbc_clf = lgb.train(
    best_lgbc_params, # 最適なハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    # valid_sets=lgb_valid_dataset, # 検証データ
    num_boost_round=100, # boostingを行う回数
    verbose_eval=20 # ブースティング20回につき1回結果を出力
)

In [ ]:
# train-validを予測する
lgbc_train_valid_proba_y = lgbc_clf.predict(dtc_train_valid_df[lgbc_features])
lgbc_train_valid_proba_y

In [ ]:
# train-validを検証する（AUCを算出する）
lgbc_train_valid_auc_val = roc_auc_score(dtc_train_valid_df[target], lgbc_train_valid_proba_y)
lgbc_train_valid_auc_val

In [ ]:
# train-validを予測する
# 確率が0.5以上の時1と判定する
lgbc_train_valid_pred_y = np.where(lgbc_train_valid_proba_y >= 0.5, 1, 0)

# train-validを検証する（Accuracyを算出する）
lgbc_train_train_accuracy_val = accuracy_score(dtc_train_valid_df[target], lgbc_train_valid_pred_y)
lgbc_train_train_accuracy_val

### 12.5.2 予測と検証

In [ ]:
# trainを予測する
lgbc_train_proba_y = lgbc_clf.predict(dtc_train_df[lgbc_features])
lgbc_train_pred_y = np.where(lgbc_train_proba_y >= 0.5, 1, 0)

# trainを検証する
lgbc_train_accuracy_val = accuracy_score(dtc_train_df[target], lgbc_train_pred_y)
lgbc_train_auc_val = roc_auc_score(dtc_train_df[target], lgbc_train_proba_y)
lgbc_train_accuracy_val, lgbc_train_auc_val

### 12.5.3 テストデータの予測

In [ ]:
# testを予測する
lgbc_test_proba_y = lgbc_clf.predict(dtc_test_df[lgbc_features])

In [ ]:
# sample submitデータを読み込む
gi_sample_submit_df = pd.read_csv("../input/gi_sample_submit.csv")

In [ ]:
# submit向けDataFrameを作成し、列に予測確率を格納する
submit_df = dtc_test_df.copy()[["customer_id"]]
submit_df["buy_proba"] = lgbc_test_proba_y
submit_df.head()

In [ ]:
gi_sample_submit_df.shape, submit_df.shape

In [ ]:
submit_df = pd.merge(gi_sample_submit_df.drop("buy_proba", axis=1), submit_df, on="customer_id", how="left").reset_index(drop=True)
submit_df.head()

In [ ]:
submit_df.shape

In [ ]:
# outputディレクトリにsubmit⽤ファイルを出⼒する
submit_df.to_csv(f"../output/submit_lgbc.csv", encoding="utf-8", index=False)